# Calculate head rotation matricies

calculate orthonormal basis vectors of the head on each frame, stack them on top of each other to make 3x3 rotation matricies, because apparently that's how that works?!

1 - calculate `head_center_xyz` as mean of left and right ear markers
2 - define x+ as vector between `head_center_xyz` and `nose_xyz`- normalize to get `x_hat`
3 - define y+ as vector between `head_center_xyz` and `left_ear_xyz` - normalize to get `y_hat_temp` (will be replaced later, to ensure orthogonal \)
4 - define z+ as cross product of `x_hat` and `y_hat`
5 - replace `y_hat_temp` with `z_hat` crossed wtih `x_hat` (Pretty sure that makes it right handed...)

Stack x_hat, y_hat, and z_hat on top of each other, and that's your head rotation matrix on that frame


install requirements to Python kernel (I think a 'kernel' is roughly equivalent to a conda environment, i.e. telling the system which version of `python.exe` you want to use to evaluate this Python code )

In [1]:
#I should specify the versions to ensure future stability
!pip install numpy
!pip install matplotlib
!pip install pyqt5
!pip install keyboard

In [2]:

import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
import keyboard

In [3]:
debug = True
# %matplotlib widget
import matplotlib.pyplot as plt
# %matplotlib widget

%matplotlib qt 
#pops out a window for matplot lib plots. It'd be cool to figure out how to play animations inline

In [4]:

### get yr paths right
session_id = 'sesh_2022-02-15_11_54_28_pupil_maybe'

freemocap_data_folder = Path('C:/Users/jonma/Dropbox/FreeMoCapProject/FreeMocap_Data')

session_folder_path = freemocap_data_folder / session_id

data_arrays_path = session_folder_path / 'DataArrays'

mediapipe_skeleton_data_path = data_arrays_path /  'mediaPipeSkel_3d_smoothed.npy'


 ### load in data

- full body 3d kinematic data - `mediapipe_skeleton_frame_mar_xyz`
    - type: numpy array
    - dimensions: [frame_number, tracked_point_number, dimension (xyz)]



In [5]:
mediapipe_skeleton_fr_mar_xyz = np.load(mediapipe_skeleton_data_path)

In [6]:
nose_id= 0
right_ear_id = 8
left_ear_id = 7

nose_xyz = mediapipe_skeleton_fr_mar_xyz[:, nose_id, :]
right_ear_xyz = mediapipe_skeleton_fr_mar_xyz[:, right_ear_id, :]
left_ear_xyz = mediapipe_skeleton_fr_mar_xyz[:, left_ear_id, :]

head_center_xyz = (right_ear_xyz + left_ear_xyz) / 2    

In [7]:
fig = plt.figure(num=123)
ax = fig.add_subplot(111)
ax.plot(nose_xyz[:,0], label='nose_x')
ax.plot(nose_xyz[:,1], label='nose_y')
ax.plot(nose_xyz[:,2], label='nose_z')
ax.plot(right_ear_xyz[:,0], label='right_ear_x')
ax.plot(right_ear_xyz[:,1], label='right_ear_y')
ax.plot(right_ear_xyz[:,2], label='right_ear_z')
ax.plot(left_ear_xyz[:,0], label='left_ear_x')
ax.plot(left_ear_xyz[:,1], label='left_ear_y')
ax.plot(left_ear_xyz[:,2], label='left_ear_z')
ax.plot(head_center_xyz[:,0], label='head_center_x')
ax.plot(head_center_xyz[:,1], label='head_center_y')
ax.plot(head_center_xyz[:,2], label='head_center_z')
ax.legend()
plt.show()
plt.pause(.1)

zero everything out, aka pin it to the origin, aka "subtract the position of the thing you want to be the origin from everything else", aka "do the thing that makes the X,Y,Z coordinates of the thing that should be (0,0,0) equal to (0,0,0)"

In [8]:
z_nose_xyz = nose_xyz - head_center_xyz
z_nose_x = z_nose_xyz[:,0]
z_nose_y = z_nose_xyz[:,1]
z_nose_z = z_nose_xyz[:,2]

z_left_ear_xyz = left_ear_xyz - head_center_xyz
z_left_ear_x = z_left_ear_xyz[:,0]
z_left_ear_y = z_left_ear_xyz[:,1]
z_left_ear_z = z_left_ear_xyz[:,2]

z_nose_xyz

array([[         nan,          nan,          nan],
       [         nan,          nan,          nan],
       [         nan,          nan,          nan],
       ...,
       [ 12.47374782,  69.0853272 , -22.37502357],
       [ 14.10629544,  60.84139366,  -6.04679083],
       [ 25.76475466,  63.75538109, -21.2651289 ]])

Normalize so they are unit vectors

In [9]:
z_norm_nose_xyz = z_nose_xyz / np.linalg.norm(z_nose_xyz, axis=1)[:,None]
z_norm_nose_x = z_norm_nose_xyz[:,0]
z_norm_nose_y = z_norm_nose_xyz[:,1]
z_norm_nose_z = z_norm_nose_xyz[:,2]

z_norm_left_ear_xyz = z_left_ear_xyz / np.linalg.norm(z_left_ear_xyz, axis=1)[:,None]
z_norm_left_ear_x = z_norm_left_ear_xyz[:,0]
z_norm_left_ear_y = z_norm_left_ear_xyz[:,1]
z_norm_left_ear_z = z_norm_left_ear_xyz[:,2]

define x_hat as the zero'd out nose vector (i.e. the +X axis of the head starts at the head center and goes out the person's nose)

In [10]:
x_hat_xyz = z_norm_nose_xyz
x_hat_x = x_hat_xyz[:,0]
x_hat_y = x_hat_xyz[:,1]
x_hat_z = x_hat_xyz[:,2]

x_hat_xyz

array([[        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       ...,
       [ 0.16929195,  0.93761632, -0.30367067],
       [ 0.22481111,  0.96962534, -0.09636731],
       [ 0.35795511,  0.8857668 , -0.29544087]])

define z as x cross y

In [11]:
z_hat_xyz = np.cross(x_hat_xyz, z_norm_left_ear_xyz) #create `z_hat` a vector perpendicular to `x_hat` and the vector from the nose to the left ear(i.e. pseudo y_hat)
z_hat_xyz = z_hat_xyz / np.linalg.norm(z_hat_xyz, axis=1)[:,None] #not sure if this needs normalizing, but it couldn't hurt
z_hat_x = z_hat_xyz[:,0]
z_hat_y = z_hat_xyz[:,1]
z_hat_z = z_hat_xyz[:,2]

z_hat_xyz

array([[        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       ...,
       [ 0.07844496, -0.31995954, -0.9441781 ],
       [ 0.23993877, -0.15094018, -0.95898199],
       [ 0.15626882, -0.36877272, -0.91628966]])

define y_hat as z_hat cross with x_hat (this ensures that they are all orthogonal, where as the original vector to the left ear might not have been  )

In [12]:
y_hat_xyz = np.cross(z_hat_xyz, z_norm_nose_xyz)
y_hat_xyz = y_hat_xyz / np.linalg.norm(y_hat_xyz, axis=1)[:,None] #not sure if this needs normalizing, but it couldn't hurt
y_hat_x = y_hat_xyz[:,0]
y_hat_y = y_hat_xyz[:,1]
y_hat_z = y_hat_xyz[:,2]

y_hat_xyz

array([[        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       ...,
       [ 0.98243912, -0.13602032,  0.12771785],
       [ 0.94439894, -0.19246755,  0.26658374],
       [ 0.9205695 , -0.28182237,  0.27042181]])

create head rotation matricies by stacking x_hat_xyz, y_hat_xyz, and z_hat_xyz

In [13]:
head_rotation_matricies = [np.dstack((this_x_hat_xyz, this_y_hat_xyz, this_z_hat_xyz))
                            for this_x_hat_xyz, this_y_hat_xyz, this_z_hat_xyz in zip(x_hat_xyz, y_hat_xyz, z_hat_xyz)]

print(len(head_rotation_matricies))
print(head_rotation_matricies[0])

4916
[[[nan nan nan]
  [nan nan nan]
  [nan nan nan]]]


make a point in head space and rotate it with the head rotation matricies, as a test of these strange mathemagics

In [14]:
test_point_xyz_og = np.array((1,1,1))
test_point_xyz_rot = np.empty(z_hat_xyz.shape)


for frame_number  in range(len(head_rotation_matricies)):
    test_point_xyz_rot[frame_number,:] = head_rotation_matricies[frame_number] @ test_point_xyz_og


test_point_xyz_rot

array([[        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       ...,
       [ 1.23017602,  0.48163646, -1.12013092],
       [ 1.40914882,  0.62621761, -0.78876556],
       [ 1.43479342,  0.23517171, -0.94130872]])

Visualize results! Press ESC to quit

In [15]:
if debug:
        plt.close('all')
        fig = plt.figure(num=124)
        fig.suptitle('Press ESC to Quit')
        ax = fig.add_subplot(111, projection='3d')

        ax_range = 2


        for frame_number in range(1200, z_nose_y.shape[0]):
                ax.clear()


                #plot head orthonormal basis vectors
                ax.plot(x_hat_x[frame_number-30:frame_number],
                        x_hat_y[frame_number-30:frame_number],
                        x_hat_z[frame_number-30:frame_number], 'r-')

                ax.plot(y_hat_x[frame_number-30:frame_number],
                        y_hat_y[frame_number-30:frame_number],
                        y_hat_z[frame_number-30:frame_number], 'g-')

                ax.plot(z_hat_x[frame_number-30:frame_number],
                        z_hat_y[frame_number-30:frame_number],
                        z_hat_z[frame_number-30:frame_number], 'b-')
                
                ax.plot([0, x_hat_x[frame_number]],
                        [0, x_hat_y[frame_number]], 
                        [0, x_hat_z[frame_number]],  'r-o',label='x_hat')

                ax.plot([0, y_hat_x[frame_number]],
                        [0, y_hat_y[frame_number]],
                        [0, y_hat_z[frame_number]],  'g-o',label='y_hat')
                        
                ax.plot([0, z_hat_x[frame_number]],
                        [0, z_hat_y[frame_number]],
                        [0, z_hat_z[frame_number]],  'b-o',label='z_hat')
                # original nose and ear directions (ear should be slightly diff from y_hat)]\
                ax.plot([0, z_nose_x[frame_number]],
                        [0, z_nose_y[frame_number]], 
                        [0, z_nose_z[frame_number]],  'r:o',label='z_nose')

                ax.plot([0, z_left_ear_x[frame_number]],
                        [0, z_left_ear_y[frame_number]],
                        [0, z_left_ear_z[frame_number]],  'g:o',label='z_left_ear')

                # plot test point rotated with head rotation matricies

                ax.plot(test_point_xyz_rot[frame_number-30:frame_number, 0],
                        test_point_xyz_rot[frame_number-30:frame_number, 1],
                        test_point_xyz_rot[frame_number-30:frame_number, 2], 'k-')
                
                ax.plot([0, test_point_xyz_rot[frame_number,0]],
                        [0, test_point_xyz_rot[frame_number,1]],
                        [0, test_point_xyz_rot[frame_number,2]],  'k-o',label='test point')




                ax.set_xlim([-ax_range, ax_range])
                ax.set_ylim([-ax_range, ax_range])
                ax.set_zlim([-ax_range, ax_range])
                ax.legend()
                plt.pause(.01)
                if keyboard.is_pressed('esc'):  
                        print('You Pressed Escape!')
                        break  # finishing the loop



You Pressed Escape!


In [16]:
head_rotation_matricies_fr_row_col = np.squeeze(np.asarray(head_rotation_matricies)) #convert list to array and give a friendly squeeze to pop the extra 'singleton' dimension
head_rotation_matricies_fr_row_col.shape

(4916, 3, 3)

In [17]:
save_head_rot_path = data_arrays_path / 'mediaPipeSkel_3d_head_rotation_matricies_fr_row_col.npy'
np.save(save_head_rot_path, head_rotation_matricies_fr_row_col)